This notebook is me trying to estimate the probability of echo-call overlap in the 2  and 3 bat cases. An overlap is defined in the sense of two sounds having an overlap in time, however small eg. even if sound A is from 0-3ms and sound B is from 3-6 ms, this is also considered an overlap. Essentially, it mimics the case of temporal masking, and is thus a worst-case scenario type assumption. 

The simulations place *N* sounds of given duration into a call+inter-pulse length window independently, and then check for overlaps among the sounds. For a given *M* runs, the number of runs with at least one overlap are counted and this rate is reported as the probability rate. 

Original date of notebook creation: 2020-12-26

Author: Thejasvi Beleyur

In [1]:
import joblib
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
plt.rcParams['agg.path.chunksize'] = 10000
import numpy as np 
np.random.seed(82319)
import pandas as pd

In [2]:
%matplotlib notebook

Let's recreate a situation where a tFM of 3.4 ms is emitted every 40ms. The 40 ms includes a typical inter-call-interval + the CF component duration too. 

In [3]:
tfm_duration = 3.4
other_duration = 40

p_occurence = tfm_duration/(tfm_duration+other_duration)

In [4]:
p_overlap = p_occurence**2
print(p_occurence, p_overlap)

0.07834101382488479 0.006137314447110789


### Checking if this actually holds up to what what I'm thinking it's telling me. 
Does the 'analytical' solution actually match what I'm thinking happens. Is this the probability of a 'perfect' overlap, or is it any kind of overlap.. including the slightest 'touching' of events? Let us simulate the situation and proceed. 

In [5]:
def create_one_sound_in_time(**kwargs):
    '''
    produces one sound as a set of time points that it occupies
    '''
    sound_durn = kwargs['sound_durn']
    other_durn = kwargs['other_durn']
    resolution = kwargs.get('resolution',.1)
    
    one_cycle_durn = sound_durn+other_durn
    time_points = np.arange(0,other_durn-sound_durn+resolution,resolution)
    sound_start = np.random.choice(time_points)
    sound_end = sound_start+sound_durn
    sound_set = set(np.arange(sound_start,sound_end+resolution,resolution))
    return sound_set

def create_n_sounds(num_sounds,**kwargs):
    '''
    Creates N sounds as sets with the time-points that the sounds occupy
    '''
    all_sounds = []
    for each in range(num_sounds):
        all_sounds.append(create_one_sound_in_time(**kwargs))
    return all_sounds

def check_for_overlap(n_sounds):
    '''
    Sound overlaps are checked by a brute-force approach that checks 
    for intersections between the tiem-points occupied by all set-pairs. 
    
    eg. if 3 sounds, A,B and C are there, the AB, AC, pairs are 
    checked until an overlap is found. The search ends whne the first overlap 
    is found between sounds. 
    
    The first sound is arbitrarily chosen to be the 'target' sound. 
    '''
    overlap_found = False
    
    
    overlaps  = [len(n_sounds[0].intersection(n_sounds[i]))>0 for i in range(1,len(n_sounds))]
    if np.sum(overlaps)>0:
        return True
    else:
        return False

def prob_sound_overlaps(n_sounds,**kwargs):
    '''
    Calculates the probability of overlap between any two sounds. 
    '''
    num_trials = kwargs['num_trials']
    num_overlaps = 0
    for i in range(num_trials):
        sounds = create_n_sounds(n_sounds,**kwargs)

        overlap_seen = check_for_overlap(sounds)
        if overlap_seen:
            num_overlaps += 1 
    return num_overlaps/num_trials

### For 2 bats

In [6]:
call_durn = 3.4
num_trials = 20000
num_sounds = 2 
silence_between_durns = [40,50]
overlap_data = {}
overlap_data['call_durn'] = np.tile(call_durn, len(silence_between_durns))
overlap_data['inter_tfm_durn'] = silence_between_durns
overlap_data['tfm_overlap_prob'] = []
overlap_data['num_bats'] = np.tile(num_sounds,len(silence_between_durns))
for silence_between in silence_between_durns:
    # run 4  parallel calculations and get the median probability 
    overlap_probs = Parallel(n_jobs=4)(delayed(prob_sound_overlaps)(num_sounds,sound_durn=call_durn,other_durn=silence_between,num_trials=num_trials)for i in range(4))
    print(f'for silence{silence_between}, the median overlap probability is: {np.median(overlap_probs)}')
    overlap_data['tfm_overlap_prob'].append(np.median(overlap_probs))


for silence40, the median overlap probability is: 0.0211
for silence50, the median overlap probability is: 0.015525


### For the 3 bat situation

In [7]:

num_sounds = 3
overlap_data3 = {}
overlap_data3['call_durn'] = np.tile(call_durn, len(silence_between_durns))
overlap_data3['inter_tfm_durn'] = silence_between_durns
overlap_data3['tfm_overlap_prob'] = []
overlap_data3['num_bats'] = np.tile(num_sounds,len(silence_between_durns))
for silence_between in silence_between_durns:
    # run 4  parallel calculations and get the median probability 
    overlap_probs = Parallel(n_jobs=4)(delayed(prob_sound_overlaps)(num_sounds,sound_durn=call_durn,other_durn=silence_between,num_trials=num_trials)for i in range(4))
    print(f'for silence{silence_between}, the median overlap probability is: {np.median(overlap_probs)}')
    overlap_data3['tfm_overlap_prob'].append(np.median(overlap_probs))


for silence40, the median overlap probability is: 0.042800000000000005
for silence50, the median overlap probability is: 0.030324999999999998


In [8]:
overlap_probs = pd.concat((pd.DataFrame(overlap_data), pd.DataFrame(overlap_data3))).reset_index(drop=True)
overlap_probs

,call_durn,inter_tfm_durn,tfm_overlap_prob,num_bats
0,3.4,40,0.021100,2
1,3.4,50,0.015525,2
2,3.4,40,0.042800,3
3,3.4,50,0.030325,3


In [9]:
overlap_probs.to_csv('2-3bat_tfm_overlap_prob.csv')